In [1]:
import pandas as pd
import dask.dataframe as dd
import praw
from dask.diagnostics import ProgressBar
from tqdm import tqdm
import hashlib
from PIL import Image
import requests
from io import BytesIO
from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.spark.set_environ import set_azure_env
import os
import torch
from datetime import datetime
import pandas as pd
from pmaw import PushshiftAPI

set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from shared_code.utility.scripts.blip_caption import BlipCaption

pbar = ProgressBar()
pbar.register()

tqdm.pandas()
tqdm.pandas(desc="global")

from tqdm.dask import TqdmCallback
cb = TqdmCallback(desc="global")
cb.register()

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [2]:
%%time

subs = ["SFWRedheads","sfwpetite","SFWNextDoorGirls","CityPorn","EarthPorn","memes","spaceporn","trippinthroughtime","fatsquirrelhate","oldladiesbakingpies","itookapicture"]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
display(extant_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17020,11ajmf8,itookapicture,caue_st,ITAP of a little spider,there is a small insect on the red flowers in ...,2ea9dabac6e5e29fcabba6bb4e67d69c,/r/itookapicture/comments/11ajmf8/itap_of_a_li...,https://i.redd.it/7hda3drs64ka1.jpg,2ea9dabac6e5e29fcabba6bb4e67d69c.jpg,D:\data\images\itookapicture\2ea9dabac6e5e29fc...,D:\data\images\itookapicture\thumbnail\2ea9dab...,True,False
17021,11aijs3,itookapicture,Eirik100,ITAP of an old wall,a close up of a red and white fire hydrant wit...,3f2168df91d1ad5ee78e5cf14c947045,/r/itookapicture/comments/11aijs3/itap_of_an_o...,https://i.imgur.com/h17lyth.jpg,3f2168df91d1ad5ee78e5cf14c947045.jpg,D:\data\images\itookapicture\3f2168df91d1ad5ee...,D:\data\images\itookapicture\thumbnail\3f2168d...,True,False
17022,11aies8,itookapicture,WittilyOrder,ITAP of a buck and his bird in the morning mist,arafed image of a deer in the foggy morning,6230a5e6312b05a2728785e10c5f59bb,/r/itookapicture/comments/11aies8/itap_of_a_bu...,https://i.redd.it/gkugpkb9d2ka1.jpg,6230a5e6312b05a2728785e10c5f59bb.jpg,D:\data\images\itookapicture\6230a5e6312b05a27...,D:\data\images\itookapicture\thumbnail\6230a5e...,True,False
17023,11aibqx,itookapicture,Max_McAlpin,ITAP a Mountain in Niseko Japan,skiers on a ski slope with a mountain in the b...,2e670a882739afed75ed8c0883e89c78,/r/itookapicture/comments/11aibqx/itap_a_mount...,https://i.redd.it/b8k6tgc0u3ka1.jpg,2e670a882739afed75ed8c0883e89c78.jpg,D:\data\images\itookapicture\2e670a882739afed7...,D:\data\images\itookapicture\thumbnail\2e670a8...,True,False


CPU times: total: 672 ms
Wall time: 1.38 s


In [4]:
reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

api: PushshiftAPI = PushshiftAPI(praw=reddit, num_workers=12)
start_date: int = int(datetime.strptime('2022-01-01', '%Y-%m-%d').timestamp())

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [5]:
def filter_by_image_post(item) -> bool:
	return not (item['selftext'].__contains__('[removed]') or item['selftext'].__contains__('[deleted]') and not item[
		'url'].endswith('.jpg'))

In [6]:
def fetch_image(x: object) -> object:
	try:
		url = x['original_url']
		subreddit = x['subreddit']
		response = requests.get(url)
		md5 = hashlib.md5(response.content).hexdigest()
		os.makedirs(f"D:\\data\\images\\{subreddit}", exist_ok=True)
		out_path = f"D:\\data\\images\\{subreddit}\\{md5}.jpg"

		if os.path.exists(out_path):
			return out_path

		if md5 in list(extant_data['hash']):
			return out_path

		if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
			raw_image = Image.open(BytesIO(response.content))
			raw_image.save(out_path)
			raw_image.close()
			return out_path
		else:
			return ""
	except Exception as e:
		# print(e)
		return ""

In [7]:
def make_thumbnail(x: object) -> str:
	image_path = x['path']
	md5 = x['hash']
	subreddit = x['subreddit']
	os.makedirs(f"D:\\data\\images\\{subreddit}\\thumbnail", exist_ok=True)
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5}.jpg"
	if not os.path.exists(image_path):
		return ""
	if os.path.exists(out_path):
		return out_path
	img = Image.open(image_path)
	try:
		copied_image = img.copy()
		result_copy = copied_image.resize((512, 512))
		result_copy.save(out_path)
		result_copy.close()
		img.close()
		return out_path
	except Exception as e:
		# print(e)
		return ""
	finally:
		img.close()

In [8]:
def exists(x: object) -> bool:
	try:
		image_path = os.path.exists(x['path'])
		thumbnail_path = os.path.exists(x['thumbnail_path'])
		return image_path and thumbnail_path
	except Exception as e:
		# print(e)
		return False

In [9]:
def split_hash(x: object) -> str:
	try:
		name = x['image_name']
		return name.split('.')[0]
	except Exception as e:
		# print(e)
		return ""

In [10]:
def caption_image(x: object, blip_captioning: BlipCaption) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if len(resulting_caption) > 5:
			return resulting_caption

		resulting_caption = blip_captioning.caption_image(path)
		return resulting_caption

	except Exception as e:
		return ""
	finally:
		pass

In [11]:
def get_image_name(x: object) -> str:
	try:
		path = x['path']
		if path == "":
			return ""
		return os.path.basename(path)
	except Exception as e:
		# print(e)
		return ""

In [12]:
%%time

all_posts = []

for item in subs:
	posts = api.search_submissions(search_window=365, subreddit=item, limit=1000, filter_fn=filter_by_image_post, mem_safe=True, cache_dir='D:\\cache\\')
	[ all_posts.append(post) for post in posts ]

CPU times: total: 38.9 s
Wall time: 9min 48s


In [13]:
%%time

df = pd.DataFrame([item for item in all_posts])
display(df)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,_fetched,_comments_by_id,is_gallery,media_metadata,gallery_data,crosspost_parent_list,crosspost_parent,link_flair_template_id,author_cakeday,poll_data
0,2048,confidence,<praw.reddit.Reddit object at 0x0000021E6F586C40>,None,SFWRedheads,,t2_fctwly2q,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2048,confidence,<praw.reddit.Reddit object at 0x0000021E6F586C40>,None,SFWRedheads,,t2_g7saqeok,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2048,confidence,<praw.reddit.Reddit object at 0x0000021E6F586C40>,None,SFWRedheads,,t2_jb4tww7x,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2048,confidence,<praw.reddit.Reddit object at 0x0000021E6F586C40>,None,SFWRedheads,,t2_t3oiarza,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2048,confidence,<praw.reddit.Reddit object at 0x0000021E6F586C40>,None,SFWRedheads,,t2_4xbd29su,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7514,2048,confidence,<praw.reddit.Reddit object at 0x0000021E7758B490>,None,itookapicture,,t2_15tmjm,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,0beb6f5e-621e-11e6-b150-0ed58fa42e6b,NaN,NaN
7515,2048,confidence,<praw.reddit.Reddit object at 0x0000021E7758B490>,None,itookapicture,,t2_iwawx,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7516,2048,confidence,<praw.reddit.Reddit object at 0x0000021E7758B490>,None,itookapicture,,t2_6dksijry,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7517,2048,confidence,<praw.reddit.Reddit object at 0x0000021E7758B490>,None,itookapicture,,t2_21sya524,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


CPU times: total: 703 ms
Wall time: 1.02 s


In [14]:
%%time

initial = pd.DataFrame({}, columns=schema.names)
initial['id'] = df['id']
initial['subreddit'] = df['subreddit']
initial['author'] = df['author']
initial['title'] = df['title']
initial['caption'] = ""
initial['hash'] = ""
initial['permalink'] = df['permalink']
initial['original_url'] = df['url']
initial['image_name'] = ""
initial['path'] = ""
initial['thumbnail_path'] = ""
initial['exists'] = False
initial['curated'] = False

filtered_initial = initial.where(initial['original_url'].str.endswith('.jpg')).dropna(how='all').reset_index().drop('index', axis=1)

display(filtered_initial)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,yx16dt,SFWRedheads,Iambatmansmom38,"Do you like to read, favorite book?",,,/r/SFWRedheads/comments/yx16dt/do_you_like_to_...,https://i.redd.it/v0u97to9ae0a1.jpg,,,,False,False
1,ywyq7p,SFWRedheads,xangelxfacexx,embrace my cuteness!,,,/r/SFWRedheads/comments/ywyq7p/embrace_my_cute...,https://i.imgur.com/24D9QJU.jpg,,,,False,False
2,ywyol9,SFWRedheads,BeeBinisky,First Snow Fall Has Me Excited ! ❄⛄,,,/r/SFWRedheads/comments/ywyol9/first_snow_fall...,https://i.redd.it/qg97b6r9vd0a1.jpg,,,,False,False
3,ywxa1i,SFWRedheads,badddjuju,good morning,,,/r/SFWRedheads/comments/ywxa1i/good_morning/,https://i.redd.it/rkbwiuqvmd0a1.jpg,,,,False,False
4,ywsrq0,SFWRedheads,SS_A_D,I had to take a selfie before going to the gym...,,,/r/SFWRedheads/comments/ywsrq0/i_had_to_take_a...,https://i.redd.it/hecmor3src0a1.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5624,ywjp42,itookapicture,leflore4,ITAP - Of coach teaching young men to stand st...,,,/r/itookapicture/comments/ywjp42/itap_of_coach...,https://i.redd.it/8e4ku12iq80a1.jpg,,,,False,False
5625,ywjli7,itookapicture,Naj183,ITAP of a Turkey Vulture spreading its wings,,,/r/itookapicture/comments/ywjli7/itap_of_a_tur...,https://i.redd.it/35zgwn967a0a1.jpg,,,,False,False
5626,ywjew1,itookapicture,ExaminationHopeful36,ITAP of my cat looking out of the window,,,/r/itookapicture/comments/ywjew1/itap_of_my_ca...,https://i.redd.it/orn7pcse5a0a1.jpg,,,,False,False
5627,ywjeai,itookapicture,retrospektor,ITAP of an empty beer bottle,,,/r/itookapicture/comments/ywjeai/itap_of_an_em...,https://i.redd.it/oyuscvpg5a0a1.jpg,,,,False,False


CPU times: total: 172 ms
Wall time: 265 ms


In [15]:
%%time

new_entries = filtered_initial.where(~filtered_initial['id'].isin(extant_data['id'])).dropna(how='all').reset_index().drop('index', axis=1)

display(new_entries)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,yx16dt,SFWRedheads,Iambatmansmom38,"Do you like to read, favorite book?",,,/r/SFWRedheads/comments/yx16dt/do_you_like_to_...,https://i.redd.it/v0u97to9ae0a1.jpg,,,,False,False
1,ywyq7p,SFWRedheads,xangelxfacexx,embrace my cuteness!,,,/r/SFWRedheads/comments/ywyq7p/embrace_my_cute...,https://i.imgur.com/24D9QJU.jpg,,,,False,False
2,ywyol9,SFWRedheads,BeeBinisky,First Snow Fall Has Me Excited ! ❄⛄,,,/r/SFWRedheads/comments/ywyol9/first_snow_fall...,https://i.redd.it/qg97b6r9vd0a1.jpg,,,,False,False
3,ywxa1i,SFWRedheads,badddjuju,good morning,,,/r/SFWRedheads/comments/ywxa1i/good_morning/,https://i.redd.it/rkbwiuqvmd0a1.jpg,,,,False,False
4,ywsrq0,SFWRedheads,SS_A_D,I had to take a selfie before going to the gym...,,,/r/SFWRedheads/comments/ywsrq0/i_had_to_take_a...,https://i.redd.it/hecmor3src0a1.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,ywjp42,itookapicture,leflore4,ITAP - Of coach teaching young men to stand st...,,,/r/itookapicture/comments/ywjp42/itap_of_coach...,https://i.redd.it/8e4ku12iq80a1.jpg,,,,False,False
3384,ywjli7,itookapicture,Naj183,ITAP of a Turkey Vulture spreading its wings,,,/r/itookapicture/comments/ywjli7/itap_of_a_tur...,https://i.redd.it/35zgwn967a0a1.jpg,,,,False,False
3385,ywjew1,itookapicture,ExaminationHopeful36,ITAP of my cat looking out of the window,,,/r/itookapicture/comments/ywjew1/itap_of_my_ca...,https://i.redd.it/orn7pcse5a0a1.jpg,,,,False,False
3386,ywjeai,itookapicture,retrospektor,ITAP of an empty beer bottle,,,/r/itookapicture/comments/ywjeai/itap_of_an_em...,https://i.redd.it/oyuscvpg5a0a1.jpg,,,,False,False


CPU times: total: 109 ms
Wall time: 154 ms


In [16]:
%%time

new_entries_1 = new_entries.copy()

tqdm.pandas(desc="Fetch-Image")
with ProgressBar():
	new_entries_1['path'] = new_entries_1.progress_apply(lambda x: fetch_image(x), axis=1)

tqdm.pandas(desc="Get-Image-Name")
with ProgressBar():
	new_entries_1['image_name'] = new_entries_1.progress_apply(lambda x: get_image_name(x), axis=1)

tqdm.pandas(desc="Split-Hash")
with ProgressBar():
	new_entries_1['hash'] = new_entries_1.progress_apply(lambda x: split_hash(x), axis=1)

display(new_entries_1)

Fetch-Image:   0%|          | 9/3388 [00:03<20:50,  2.70it/s]

cannot write mode P as JPEG


Fetch-Image:   1%|          | 40/3388 [00:24<19:33,  2.85it/s]  

cannot write mode P as JPEG
cannot write mode P as JPEG


Fetch-Image:   4%|▍         | 129/3388 [00:57<11:59,  4.53it/s]

...

Fetch-Image:  85%|████████▌ | 2887/3388 [20:24<05:45,  1.45it/s]

cannot write mode RGBA as JPEG


Split-Hash: 100%|██████████| 3388/3388 [00:00<00:00, 22437.15it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,yx16dt,SFWRedheads,Iambatmansmom38,"Do you like to read, favorite book?",,fd91eb6f88d9a51e253c88a0c61daf53,/r/SFWRedheads/comments/yx16dt/do_you_like_to_...,https://i.redd.it/v0u97to9ae0a1.jpg,fd91eb6f88d9a51e253c88a0c61daf53.jpg,D:\data\images\SFWRedheads\fd91eb6f88d9a51e253...,,False,False
1,ywyq7p,SFWRedheads,xangelxfacexx,embrace my cuteness!,,bb4f9cbc9342c47b74e49877a6445d68,/r/SFWRedheads/comments/ywyq7p/embrace_my_cute...,https://i.imgur.com/24D9QJU.jpg,bb4f9cbc9342c47b74e49877a6445d68.jpg,D:\data\images\SFWRedheads\bb4f9cbc9342c47b74e...,,False,False
2,ywyol9,SFWRedheads,BeeBinisky,First Snow Fall Has Me Excited ! ❄⛄,,47e18745b435f4796978ad6078e2d9c6,/r/SFWRedheads/comments/ywyol9/first_snow_fall...,https://i.redd.it/qg97b6r9vd0a1.jpg,47e18745b435f4796978ad6078e2d9c6.jpg,D:\data\images\SFWRedheads\47e18745b435f479697...,,False,False
3,ywxa1i,SFWRedheads,badddjuju,good morning,,ca5a9e6b00ccfebeabbf07e236670288,/r/SFWRedheads/comments/ywxa1i/good_morning/,https://i.redd.it/rkbwiuqvmd0a1.jpg,ca5a9e6b00ccfebeabbf07e236670288.jpg,D:\data\images\SFWRedheads\ca5a9e6b00ccfebeabb...,,False,False
4,ywsrq0,SFWRedheads,SS_A_D,I had to take a selfie before going to the gym...,,427ab2dc4bc15b2ea6180f3457dacd58,/r/SFWRedheads/comments/ywsrq0/i_had_to_take_a...,https://i.redd.it/hecmor3src0a1.jpg,427ab2dc4bc15b2ea6180f3457dacd58.jpg,D:\data\images\SFWRedheads\427ab2dc4bc15b2ea61...,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,ywjp42,itookapicture,leflore4,ITAP - Of coach teaching young men to stand st...,,f9c5e68a2e6082b0827469253807647e,/r/itookapicture/comments/ywjp42/itap_of_coach...,https://i.redd.it/8e4ku12iq80a1.jpg,f9c5e68a2e6082b0827469253807647e.jpg,D:\data\images\itookapicture\f9c5e68a2e6082b08...,,False,False
3384,ywjli7,itookapicture,Naj183,ITAP of a Turkey Vulture spreading its wings,,a37c752a3ecdd654cd402280b0a3de7f,/r/itookapicture/comments/ywjli7/itap_of_a_tur...,https://i.redd.it/35zgwn967a0a1.jpg,a37c752a3ecdd654cd402280b0a3de7f.jpg,D:\data\images\itookapicture\a37c752a3ecdd654c...,,False,False
3385,ywjew1,itookapicture,ExaminationHopeful36,ITAP of my cat looking out of the window,,53f99cb828e71b8a3172fea41d1f2259,/r/itookapicture/comments/ywjew1/itap_of_my_ca...,https://i.redd.it/orn7pcse5a0a1.jpg,53f99cb828e71b8a3172fea41d1f2259.jpg,D:\data\images\itookapicture\53f99cb828e71b8a3...,,False,False
3386,ywjeai,itookapicture,retrospektor,ITAP of an empty beer bottle,,9d5acf1a16798238d606fc3646d16f75,/r/itookapicture/comments/ywjeai/itap_of_an_em...,https://i.redd.it/oyuscvpg5a0a1.jpg,9d5acf1a16798238d606fc3646d16f75.jpg,D:\data\images\itookapicture\9d5acf1a16798238d...,,False,False


CPU times: total: 12min 26s
Wall time: 25min 58s


In [17]:
%%time

new_entries_2 = new_entries_1.copy()

tqdm.pandas(desc="Create-Thumbnail")
with ProgressBar():
	new_entries_2['thumbnail_path'] = new_entries_2.progress_apply(lambda x: make_thumbnail(x), axis=1)

tqdm.pandas(desc="Checking-Exists")
with ProgressBar():
	new_entries_2['exists'] = new_entries_2.progress_apply(lambda x: exists(x), axis=1)

display(new_entries_2)

Checking-Exists: 100%|██████████| 3388/3388 [00:01<00:00, 2204.29it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,yx16dt,SFWRedheads,Iambatmansmom38,"Do you like to read, favorite book?",,fd91eb6f88d9a51e253c88a0c61daf53,/r/SFWRedheads/comments/yx16dt/do_you_like_to_...,https://i.redd.it/v0u97to9ae0a1.jpg,fd91eb6f88d9a51e253c88a0c61daf53.jpg,D:\data\images\SFWRedheads\fd91eb6f88d9a51e253...,D:\data\images\SFWRedheads\thumbnail\fd91eb6f8...,True,False
1,ywyq7p,SFWRedheads,xangelxfacexx,embrace my cuteness!,,bb4f9cbc9342c47b74e49877a6445d68,/r/SFWRedheads/comments/ywyq7p/embrace_my_cute...,https://i.imgur.com/24D9QJU.jpg,bb4f9cbc9342c47b74e49877a6445d68.jpg,D:\data\images\SFWRedheads\bb4f9cbc9342c47b74e...,D:\data\images\SFWRedheads\thumbnail\bb4f9cbc9...,True,False
2,ywyol9,SFWRedheads,BeeBinisky,First Snow Fall Has Me Excited ! ❄⛄,,47e18745b435f4796978ad6078e2d9c6,/r/SFWRedheads/comments/ywyol9/first_snow_fall...,https://i.redd.it/qg97b6r9vd0a1.jpg,47e18745b435f4796978ad6078e2d9c6.jpg,D:\data\images\SFWRedheads\47e18745b435f479697...,D:\data\images\SFWRedheads\thumbnail\47e18745b...,True,False
3,ywxa1i,SFWRedheads,badddjuju,good morning,,ca5a9e6b00ccfebeabbf07e236670288,/r/SFWRedheads/comments/ywxa1i/good_morning/,https://i.redd.it/rkbwiuqvmd0a1.jpg,ca5a9e6b00ccfebeabbf07e236670288.jpg,D:\data\images\SFWRedheads\ca5a9e6b00ccfebeabb...,D:\data\images\SFWRedheads\thumbnail\ca5a9e6b0...,True,False
4,ywsrq0,SFWRedheads,SS_A_D,I had to take a selfie before going to the gym...,,427ab2dc4bc15b2ea6180f3457dacd58,/r/SFWRedheads/comments/ywsrq0/i_had_to_take_a...,https://i.redd.it/hecmor3src0a1.jpg,427ab2dc4bc15b2ea6180f3457dacd58.jpg,D:\data\images\SFWRedheads\427ab2dc4bc15b2ea61...,D:\data\images\SFWRedheads\thumbnail\427ab2dc4...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,ywjp42,itookapicture,leflore4,ITAP - Of coach teaching young men to stand st...,,f9c5e68a2e6082b0827469253807647e,/r/itookapicture/comments/ywjp42/itap_of_coach...,https://i.redd.it/8e4ku12iq80a1.jpg,f9c5e68a2e6082b0827469253807647e.jpg,D:\data\images\itookapicture\f9c5e68a2e6082b08...,D:\data\images\itookapicture\thumbnail\f9c5e68...,True,False
3384,ywjli7,itookapicture,Naj183,ITAP of a Turkey Vulture spreading its wings,,a37c752a3ecdd654cd402280b0a3de7f,/r/itookapicture/comments/ywjli7/itap_of_a_tur...,https://i.redd.it/35zgwn967a0a1.jpg,a37c752a3ecdd654cd402280b0a3de7f.jpg,D:\data\images\itookapicture\a37c752a3ecdd654c...,D:\data\images\itookapicture\thumbnail\a37c752...,True,False
3385,ywjew1,itookapicture,ExaminationHopeful36,ITAP of my cat looking out of the window,,53f99cb828e71b8a3172fea41d1f2259,/r/itookapicture/comments/ywjew1/itap_of_my_ca...,https://i.redd.it/orn7pcse5a0a1.jpg,53f99cb828e71b8a3172fea41d1f2259.jpg,D:\data\images\itookapicture\53f99cb828e71b8a3...,D:\data\images\itookapicture\thumbnail\53f99cb...,True,False
3386,ywjeai,itookapicture,retrospektor,ITAP of an empty beer bottle,,9d5acf1a16798238d606fc3646d16f75,/r/itookapicture/comments/ywjeai/itap_of_an_em...,https://i.redd.it/oyuscvpg5a0a1.jpg,9d5acf1a16798238d606fc3646d16f75.jpg,D:\data\images\itookapicture\9d5acf1a16798238d...,D:\data\images\itookapicture\thumbnail\9d5acf1...,True,False


CPU times: total: 13min 38s
Wall time: 16min 50s


In [18]:
%%time

blip = BlipCaption(1)

CPU times: total: 1min 4s
Wall time: 1min 16s


In [ ]:
%%time

new_entries_final = new_entries_2.copy()
torch.cuda.empty_cache()

with pbar:
	ddf = dd.from_pandas(new_entries_final, npartitions=6)
	new_entries_final['caption'] = ddf.apply(lambda x: caption_image(x, blip), meta=('str', object), axis=1).compute()

torch.cuda.empty_cache()

display(new_entries_final)

[                                        ] | 0% Completed | 10.07 ms

global:   0%|          | 0/6 [00:00<?, ?it/s]

[                                        ] | 0% Completed | 21.97 sms

D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[                                        ] | 0% Completed | 56m 34ss

In [ ]:
%%time

tqdm.pandas(desc="Filtering")
writeable_entries = new_entries_final\
	.where(~new_entries_final['id'].isin(extant_data['id']))\
	.dropna(how='all')\
	.reset_index()\
	.drop('index', axis=1)

display(writeable_entries)

In [ ]:
%%time

combined_result = pd.concat([extant_data, writeable_entries])

display(combined_result)

In [ ]:
combined_result.reindex()
combined_result.to_csv('processed_raw_data.csv', index=False)
f = AzureFileStorageAdapter('data').get_file_storage()
f.put('processed_raw_data.csv','data/processed_raw_data.csv', overwrite=True)

In [ ]:
final = pd.read_csv(f.open('data/processed_raw_data.csv'))
final.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

In [ ]:
final = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(final)